## 4.6 - Método Quente-Frio (Hot and Cold)

In [1]:
import numpy as np
from time import sleep
from IPython.display import clear_output

In [2]:
weight = 0.1
# valor a ser aumentado ou diminuido do peso
# idela que seja pequeno, para que não haja muita variação
lr = 0.01
def neural_networking(input1, weight):
    prediction=input1*weight
    return prediction


In [3]:
# fazendo a predição e calculando o erro
# Mean Square Error
number_of_toes=[8.5]
win_or_lose_binary=[1] # won

input1=number_of_toes[0]
true=win_or_lose_binary[0]

pred=neural_networking(input1,weight)

error=(pred-true)**2
print(error)

0.022499999999999975


In [4]:
p_up=neural_networking(input1,weight+lr)
e_up=(p_up-true)**2
print(e_up)

0.004224999999999993


In [5]:
p_dn=neural_networking(input1,weight-lr)
e_dn=(p_dn-true)**2
print(e_dn)

0.05522499999999994


In [6]:
if((error>e_dn) | (error>e_up)):
    if e_dn < e_up:
        weight -= lr
    if e_up < e_dn:
        weight +=lr

In [7]:
pred=neural_networking(input1,weight)

error=(pred-true)**2
print(error)

0.004224999999999993


## Simplest Way

In [10]:
weight = 0.5
input1 = 0.5
goal_prediction = 0.8

#how much to move our weight in each interation
step_amount=0.001

for i in range(1101):
    clear_output(wait=True)
    prediction = input1*weight
    error = (goal_prediction-prediction) ** 2
    
    print("Error: {} Prediction: {}".format(error,prediction))
    
    #Try up
    up_prediction = input1 * (weight + step_amount)
    up_error = (goal_prediction - up_prediction) ** 2
    
    #try down
    down_prediction = input1 * (weight - step_amount)
    down_error = (goal_prediction - down_prediction) ** 2
    
    #if down is better, go down
    if(down_error < up_error):
        weight = weight - step_amount
        
    #if up is better, go up
    if(up_error < down_error):
        weight = weight + step_amount
        
    sleep(0.0001)
        

Error: 1.0799505792475652e-27 Prediction: 0.7999999999999672


In [27]:
# test the numbers of interation
i=0
input1-0.5
prediction=0

weight = 0.5
input1 = 0.5
goal_prediction = 0.8

#how much to move our weight in each interation
step_amount=0.001

while prediction<goal_prediction:
    
    clear_output(wait=True)
    prediction = input1*weight
    error = (goal_prediction-prediction) ** 2
    
    #Try up
    up_prediction = input1 * (weight + step_amount)
    up_error = (goal_prediction - up_prediction) ** 2
    
    #try down
    down_prediction = input1 * (weight - step_amount)
    down_error = (goal_prediction - down_prediction) ** 2
    
    #if down is better, go down
    if(down_error < up_error):
        weight = weight - step_amount
        
    #if up is better, go up
    if(up_error < down_error):
        weight = weight + step_amount
   
    i+=1  
    print(f"interation: {i}, Prediction: {prediction}")
    sleep(0.0001)
print(i)       

interation: 1102, Prediction: 0.8004999999999671
1102


### Calculating Both  direction  and  amount  from  error

In [32]:
weight = 0.5
goal_pred = 0.8
unput1 = 0.5

for i in range(20):
    clear_output(wait=True)
    pred=input1 * weight
    
    # pure error
    error = (pred - goal_pred) ** 2
    
    # scaling, negative reversal, and stopping
    direction_and_amount = (pred-goal_pred) *input1
    weight = weight - direction_and_amount 
    print("Error: {}, Prediction: {}".format(error, pred))
    sleep(0.1)

Error: 5.408208020258491e-06, Prediction: 0.7976744445781151


* **What is the direction_and_amount ?** 

It represents how we want to change our weight . The first part (1) is what we call "pure error" which equals ( pred - goal_pred ). (More about this below.) The second part (2) is the multiplication by the input which performs sc aling, negative reversal, and stopping, modifying the "pure error" so that it's ready to update our weight.


* **What is the "pure error"?**

It's the ( pred - goal_pred ) which indicates "the raw direction and amount that we missed". If this is a positive number th en we predicted too high and vice versa. If this is a big number then we missed by a big amount, etc.  What is "scaling, negative reversal, and stopping"?  These three attributes have the combined affect of translating our "pure error" into "the absolute amo unt that we want to change our weight ". They do so by addressing three major edge cases at which points the "pure error" is not sufficient to make a good modification to our weight.


* **What is "stopping"?** 

This is the first (and simplest) effect on our "pure error" caused by multiplying it by our input . Imagine plugging a CD player into your stereo. If you turned the volume all the way up but the CD player was off ... it simply wouldn't matter. "Stopping" addresses this in our neural network. If our input is 0, then it will force our direction_ and_amount to also be 0. We don't learn (i.e. "change the volume") when our input is 0 because there's nothing to learn. Every weight value has the same error , and moving it makes no difference because the pred is always 0. 


* **What is "negative reversal"?** 

This is probably our most difficult and important effect. Normally (when input is positive), moving our weight upward makes our prediction move upward. However, if our input is negative , then all of a sudden our weight change s directions!!! When our input is negative , then moving our weight up makes the prediction go down . It's reversed!!! How do we address this? Well, multiplying our "pure error" by our input will reverse the sign of our direction_and_amount in the event that  our input is negative. This is "negative reversal", ensuring that our weight moves in the correct direction, even if the input is negative.  

* **What is "scaling"?**

Scaling is the third effect on our "pure error" caused by multiplying it by our input . Logically, it means that if our input was big, our weight update should also be big. This is more of a "side affect" as it can often go out of control. Later, we will use alpha to address when this scaling goes out of control.  

## 4.10 - One Iteration of Gradient Descent  

In [37]:
weight = 0.1
alpha = 0.01

number_of_toes = [8.5]
win_or_lose_binary = [1] # won
input1=number_of_toes[0]
goal_pred=win_or_lose_binary[0]

pred= neural_networking(input1,weight)

error = (pred-goal_pred)**2

# delta is how much we missed?
delta=pred-goal_pred

"""
weight_delta is a measure of "how much this weight caused the
networking to miss
"""
weight_delta = input1 * delta 

weight -= weight_delta*alpha
print(weight)

0.11275


## 4.11  Learning Is Just Reducing Error  

In [43]:
weight,goal_pred,input1 = (0.0,0.8,0.5)

for i in range(4):
    error = ((input1 * weight) - goal_pred) ** 2
    delta = pred - goal_pred
    weight_delta = delta * input1
    weight = weight - weight_delta
    print("Error: {}, Prediction: {}".format(error,prediction))
    
    

Error: 0.6400000000000001, Prediction: 0.8004999999999671
Error: 0.3600000000000001, Prediction: 0.8004999999999671
Error: 0.2025, Prediction: 0.8004999999999671
Error: 0.11390625000000001, Prediction: 0.8004999999999671


## 4.21  Breaking Gradient Descent  

In [44]:
weight = 0.5  
goal_pred = 0.8  
input1 = 0.5  
for iteration in range(20):  
    pred = input1 * weight  
    error = (pred - goal_pred) ** 2  
    delta = pred - goal_pred  
    weight_delta = input1 * delta  
    weight = weight - weight_delta  
    print("Error:" + str(error) + " Prediction:" + str(pred))  

Error:0.30250000000000005 Prediction:0.25
Error:0.17015625000000004 Prediction:0.3875
Error:0.095712890625 Prediction:0.49062500000000003
Error:0.05383850097656251 Prediction:0.56796875
Error:0.03028415679931642 Prediction:0.6259765625
Error:0.0170348381996155 Prediction:0.669482421875
Error:0.00958209648728372 Prediction:0.70211181640625
Error:0.005389929274097089 Prediction:0.7265838623046875
Error:0.0030318352166796153 Prediction:0.7449378967285156
Error:0.0017054073093822882 Prediction:0.7587034225463867
Error:0.0009592916115275371 Prediction:0.76902756690979
Error:0.0005396015314842384 Prediction:0.7767706751823426
Error:0.000303525861459885 Prediction:0.7825780063867569
Error:0.00017073329707118678 Prediction:0.7869335047900676
Error:9.603747960254256e-05 Prediction:0.7902001285925507
Error:5.402108227642978e-05 Prediction:0.7926500964444131
Error:3.038685878049206e-05 Prediction:0.7944875723333098
Error:1.7092608064027242e-05 Prediction:0.7958656792499823
Error:9.614592036015323

## 4.25  Alpha In Code  

Normatizando o valor das entradas nos cálculos, para evitar a divergência na atualização dos pesos

In [56]:
weight = 0.5  
goal_pred = 0.8  
input1 = 2  
alpha = 0.10  
for iteration in range(20):  
    pred = input1 * weight  
    error = (pred - goal_pred) ** 2  
    derivative = input1 * (pred - goal_pred)
    weight = weight - (alpha * derivative)  
    print("Error:" + str(error) + " Prediction:" + str(pred))  

Error:0.03999999999999998 Prediction:1.0
Error:0.0144 Prediction:0.92
Error:0.005183999999999993 Prediction:0.872
Error:0.0018662400000000014 Prediction:0.8432000000000001
Error:0.0006718464000000028 Prediction:0.8259200000000001
Error:0.00024186470400000033 Prediction:0.815552
Error:8.70712934399997e-05 Prediction:0.8093312
Error:3.134566563839939e-05 Prediction:0.80559872
Error:1.1284439629823931e-05 Prediction:0.803359232
Error:4.062398266736526e-06 Prediction:0.8020155392
Error:1.4624633760252567e-06 Prediction:0.8012093235200001
Error:5.264868153690924e-07 Prediction:0.8007255941120001
Error:1.8953525353291194e-07 Prediction:0.8004353564672001
Error:6.82326912718715e-08 Prediction:0.8002612138803201
Error:2.456376885786678e-08 Prediction:0.8001567283281921
Error:8.842956788836216e-09 Prediction:0.8000940369969153
Error:3.1834644439835434e-09 Prediction:0.8000564221981492
Error:1.1460471998340758e-09 Prediction:0.8000338533188895
Error:4.125769919393652e-10 Prediction:0.80002031199

## Revisão

In [69]:
weight=0.5 #pesos
input1=2
goal_pred=0.8

# value to normatize the derivative in weight actualization
alpha=0.1

for i in range(20):
    # make predictions
    pred = input1*weight
    
    #calculate the error eith Minimum Square Error
    error= (pred-goal_pred)**2
    
    # Calculate the gradient descendent
    # the gradient descedent must be multiplicated for the input, to predict signal erros in direction
    derivate = input1 * (pred - goal_pred)
    
    # calculate the new weight, with derivative normalization
    # predict the divergence with large derivatives
    weight = weight - (derivate * alpha)
    
    print("Error: {}, Prediction: {}".format(error,pred))

Error: 0.03999999999999998, Prediction: 1.0
Error: 0.0144, Prediction: 0.92
Error: 0.005183999999999993, Prediction: 0.872
Error: 0.0018662400000000014, Prediction: 0.8432000000000001
Error: 0.0006718464000000028, Prediction: 0.8259200000000001
Error: 0.00024186470400000033, Prediction: 0.815552
Error: 8.70712934399997e-05, Prediction: 0.8093312
Error: 3.134566563839939e-05, Prediction: 0.80559872
Error: 1.1284439629823931e-05, Prediction: 0.803359232
Error: 4.062398266736526e-06, Prediction: 0.8020155392
Error: 1.4624633760252567e-06, Prediction: 0.8012093235200001
Error: 5.264868153690924e-07, Prediction: 0.8007255941120001
Error: 1.8953525353291194e-07, Prediction: 0.8004353564672001
Error: 6.82326912718715e-08, Prediction: 0.8002612138803201
Error: 2.456376885786678e-08, Prediction: 0.8001567283281921
Error: 8.842956788836216e-09, Prediction: 0.8000940369969153
Error: 3.1834644439835434e-09, Prediction: 0.8000564221981492
Error: 1.1460471998340758e-09, Prediction: 0.800033853318889